In [1]:
import os
# Display current working directory
print(os.getcwd())
# To make sure opencv imports .exr files
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
# If the current directory is not WDSS, then set it to one level up
if os.getcwd()[-4:] != 'WDSS':
    os.chdir('..')
print(os.getcwd())

c:\Dev\MinorProject\WDSS\jupyter_notebooks
c:\Dev\MinorProject\WDSS


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

from typing import List, Tuple, Dict

from utils.image_utils import ImageUtils
from config import device, Settings
from commons import initialize

from network.dataset import *
import tqdm

In [3]:
# Import enum
from enum import Enum

class GBufferTypes(Enum):
    NORMAL_SPECULAR = "NormalSpecular"
    BASE_COLOR_DEPTH = "BaseColorDepth"
    PRETONEMAP_METALLIC = "PretonemapMetallic"
    MV_ROUGHNESS_NoV = "MV_Roughness_NOV"

In [4]:
def pack_normal_specular(path: str, frame_no: int):
    normal_path = path + "WorldNormal"
    specular_path = path + "Specular"
    save_path = path + GBufferTypes.NORMAL_SPECULAR.value
    idx = "." + str(frame_no).zfill(4) + ".exr"

    normal = ImageUtils.load_exr_image_opencv(normal_path + idx)[:, :, :3]
    specular = ImageUtils.load_exr_image_opencv(specular_path + idx)[:, :, :1]

    packed = np.concatenate((normal, specular), axis=2)

    ImageUtils.save_exr_image_opencv(packed, save_path + idx)

    # Delete the normal and specular images
    # os.remove(normal_path + idx)
    # os.remove(specular_path + idx)

def pack_base_color_depth(path: str, frame_no: int):
    base_color_path = path + "BaseColor"
    depth_path = path + "SceneDepth"
    save_path = path + GBufferTypes.BASE_COLOR_DEPTH.value
    idx = "." + str(frame_no).zfill(4) + ".exr"

    base_color = ImageUtils.load_exr_image_opencv(base_color_path + idx)[:, :, :3]
    depth = ImageUtils.load_exr_image_opencv(depth_path + idx)[:, :, :1]

    packed = np.concatenate((base_color, depth), axis=2)

    ImageUtils.save_exr_image_opencv(packed, save_path + idx)

    # Delete the base color and depth images
    # os.remove(base_color_path + idx)
    # os.remove(depth_path + idx)

def pack_pretonemapped_metallic(path: str, frame_no: int):
    pretonemapped_path = path + "PreTonemapHDRColor"
    metallic_path = path + "Metallic"
    save_path = path + GBufferTypes.PRETONEMAP_METALLIC.value
    idx = "." + str(frame_no).zfill(4) + ".exr"

    pretonemapped = ImageUtils.load_exr_image_opencv(pretonemapped_path + idx)[:, :, :3]
    metallic = ImageUtils.load_exr_image_opencv(metallic_path + idx)[:, :, :1]

    packed = np.concatenate((pretonemapped, metallic), axis=2)

    ImageUtils.save_exr_image_opencv(packed, save_path + idx)

    # Delete the pretonemapped and metallic images
    # os.remove(pretonemapped_path + idx)
    # os.remove(metallic_path + idx)

def pack_mv_roughness_nov(path: str, frame_no: int):
    mv_path = path + "MotionVector"
    roughness_path = path + "Roughness"
    nov_path = path + "NoV"
    save_path = path + GBufferTypes.MV_ROUGHNESS_NoV.value
    idx = "." + str(frame_no).zfill(4) + ".exr"

    mv = ImageUtils.load_exr_image_opencv(mv_path + idx)[:, :, :2]
    roughness = ImageUtils.load_exr_image_opencv(roughness_path + idx)[:, :, :1]
    nov = ImageUtils.load_exr_image_opencv(nov_path + idx)[:, :, :1]

    packed = np.concatenate((mv, roughness, nov), axis=2)

    ImageUtils.save_exr_image_opencv(packed, save_path + idx)

    # Delete the mv, roughness and nov images
    # os.remove(mv_path + idx)
    # os.remove(roughness_path + idx)
    # os.remove(nov_path + idx)

In [10]:
def downsample(image: np.ndarray, target_size: Tuple[int, int]) -> np.ndarray:
    return cv2.resize(image, target_size, interpolation=cv2.INTER_NEAREST_EXACT)

def downsample_and_save_single(source_path: str, target_path: str, img_name: str, target_sizes: List[Tuple[str, Tuple[int, int]]]):
    image = ImageUtils.load_exr_image_opencv(source_path + img_name)
    for target_size in target_sizes:
        downsampled = downsample(image, target_size[1])
        ImageUtils.save_exr_image_opencv(downsampled, target_path + target_size[0] + "\\" + img_name)

def downsample_and_save_frame(source_path: str, target_path: str, target_sizes: List[Tuple[str, Tuple[int, int]]], frame_no: int):
    idx = "." + str(frame_no).zfill(4) + ".exr"
    
    for gb_type in GBufferTypes:
        downsample_and_save_single(source_path, target_path, gb_type.value + idx, target_sizes)

In [11]:
import concurrent.futures
import tqdm

def pack(path: str, total_frames: int):
    def pack_frame(i):
        pack_normal_specular(path, i)
        pack_base_color_depth(path, i)
        pack_pretonemapped_metallic(path, i)
        pack_mv_roughness_nov(path, i)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm.tqdm(executor.map(pack_frame, range(total_frames)), total=total_frames))

def downsample_and_save(source_path: str, target_path: str, target_sizes: List[Tuple[str, Tuple[int, int]]], total_frames: int):
    # Create the target directory if it does not exist
    for target_size in target_sizes:
        if not os.path.exists(target_path + target_size[0]):
            os.makedirs(target_path + target_size[0])
    
    def downsample_frame(i):
        downsample_and_save_frame(source_path, target_path, target_sizes, i)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm.tqdm(executor.map(downsample_frame, range(total_frames)), total=total_frames))

In [7]:
path = "D:\\Dev\\MinorProjDataset\\V6\\test\\"
destination = "D:\\Dev\\MinorProjDataset\\V6\\test_downsampled\\"

In [8]:
pack(path, 54)

  0%|          | 0/54 [00:00<?, ?it/s]

100%|██████████| 54/54 [11:59<00:00, 13.33s/it]  


In [12]:
downsample_and_save(
    path,
    destination,
    [
        ("1080P", (1920, 1080)),
        ("540P", (960, 540)),
        ("360P", (640, 360)),
        ("270P", (480, 270))
    ],
    54
)

100%|██████████| 54/54 [03:04<00:00,  3.42s/it]
